In [8]:
import pandas as pd
import numpy as np
import statsmodels as sm

long = pd.read_csv('patient_longitudinal.csv', sep='\t')
long['visit_date'] = pd.to_datetime(arg=long['visit_date'], format='%Y-%m-%d')
print(long.duplicated().sum()) #checking for dupes

0


In [9]:
print(long.isna().sum()) #checking for NAs

patient_id       0
visit_date       0
bp_systolic      0
heart_rate       0
adverse_event    0
age              0
sex              0
bmi              0
smoking          0
diabetes         0
dtype: int64


In [10]:
ts = pd.DatetimeIndex(long['visit_date'])
print(ts)

DatetimeIndex(['2023-01-17', '2023-09-23', '2024-05-22', '2024-12-31',
               '2023-01-17', '2023-09-23', '2024-05-22', '2024-12-31',
               '2023-01-17', '2023-09-23',
               ...
               '2024-05-22', '2024-12-31', '2023-01-17', '2023-09-23',
               '2024-05-22', '2024-12-31', '2023-01-17', '2023-09-23',
               '2024-05-22', '2024-12-31'],
              dtype='datetime64[ns]', name='visit_date', length=4000, freq=None)


In [11]:
long.set_index('visit_date', inplace = True)


In [12]:
print(long)

            patient_id  bp_systolic  heart_rate  adverse_event   age  sex  \
visit_date                                                                  
2023-01-17           0        111.0        78.7              0  57.5  0.0   
2023-09-23           0        112.0        79.5              0  57.5  0.0   
2024-05-22           0        111.2        78.7              0  57.5  0.0   
2024-12-31           0        109.2        79.5              0  57.5  0.0   
2023-01-17           1        146.1        78.9              0  47.9  0.0   
...                ...          ...         ...            ...   ...  ...   
2024-12-31         998        136.9        77.1              0  41.4  0.0   
2023-01-17         999        140.5        96.6              1  58.6  0.0   
2023-09-23         999        140.2        98.6              0  58.6  0.0   
2024-05-22         999        139.6        99.6              0  58.6  0.0   
2024-12-31         999        138.5        99.3              1  58.6  0.0   